# Database parsing


In [2]:
import sqlite3
import os 

os.makedirs("data/databases", exist_ok=True)


In [3]:
conn = sqlite3.connect("data/databases/company.db")
cursor = conn.cursor()


In [5]:

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Employees (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        role TEXT NOT NULL,
        department TEXT NOT NULL,
        salary REAL NOT NULL
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Projects (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        budget REAL NOT NULL,
        status TEXT NOT NULL,
        lead_id INTEGER,
        FOREIGN KEY (lead_id) REFERENCES Employees(id)
    )
''')

conn.commit()

In [6]:
employees_data = [
    ('John Smith', 'Software Engineer', 'Engineering', 75000),
    ('Sarah Johnson', 'Product Manager', 'Product', 85000),
    ('Mike Davis', 'Data Scientist', 'Analytics', 80000),
    ('Emily Brown', 'UX Designer', 'Design', 70000),
    ('David Wilson', 'DevOps Engineer', 'Engineering', 78000),
    ('Lisa Garcia', 'Marketing Manager', 'Marketing', 72000),
    ('Tom Anderson', 'Senior Developer', 'Engineering', 82000),
    ('Anna Martinez', 'HR Manager', 'Human Resources', 68000),
    ('Chris Taylor', 'Sales Manager', 'Sales', 76000),
    ('Jessica Lee', 'Quality Assurance', 'Engineering', 65000)
]

cursor.executemany('''
    INSERT INTO Employees (name, role, department, salary)
    VALUES (?, ?, ?, ?)
''', employees_data)

projects_data = [
    ('E-commerce Platform', 250000, 'In Progress', 1),
    ('Mobile App Development', 180000, 'Planning', 2),
    ('Data Analytics Dashboard', 120000, 'Completed', 3),
    ('User Experience Redesign', 95000, 'In Progress', 4),
    ('Cloud Migration', 300000, 'Planning', 5),
    ('Marketing Campaign Tool', 85000, 'In Progress', 6),
    ('API Integration', 150000, 'Completed', 7),
    ('Employee Portal', 110000, 'Testing', 8),
    ('CRM System', 200000, 'In Progress', 9),
    ('Quality Automation', 75000, 'Planning', 10)
]

cursor.executemany('''
    INSERT INTO Projects (name, budget, status, lead_id)
    VALUES (?, ?, ?, ?)
''', projects_data)

conn.commit()


print("\nEmployees:")
cursor.execute("SELECT * FROM Employees")
for row in cursor.fetchall():
    print(row)

print("\nProjects:")
cursor.execute("SELECT * FROM Projects")
for row in cursor.fetchall():
    print(row)


Employees:
(1, 'John Smith', 'Software Engineer', 'Engineering', 75000.0)
(2, 'Sarah Johnson', 'Product Manager', 'Product', 85000.0)
(3, 'Mike Davis', 'Data Scientist', 'Analytics', 80000.0)
(4, 'Emily Brown', 'UX Designer', 'Design', 70000.0)
(5, 'David Wilson', 'DevOps Engineer', 'Engineering', 78000.0)
(6, 'Lisa Garcia', 'Marketing Manager', 'Marketing', 72000.0)
(7, 'Tom Anderson', 'Senior Developer', 'Engineering', 82000.0)
(8, 'Anna Martinez', 'HR Manager', 'Human Resources', 68000.0)
(9, 'Chris Taylor', 'Sales Manager', 'Sales', 76000.0)
(10, 'Jessica Lee', 'Quality Assurance', 'Engineering', 65000.0)

Projects:
(1, 'E-commerce Platform', 250000.0, 'In Progress', 1)
(2, 'Mobile App Development', 180000.0, 'Planning', 2)
(3, 'Data Analytics Dashboard', 120000.0, 'Completed', 3)
(4, 'User Experience Redesign', 95000.0, 'In Progress', 4)
(5, 'Cloud Migration', 300000.0, 'Planning', 5)
(6, 'Marketing Campaign Tool', 85000.0, 'In Progress', 6)
(7, 'API Integration', 150000.0, 'Comp

#Database Content extraction


In [7]:
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader

In [9]:
#METHOD ONE : SQLDatabaseLoader
db = SQLDatabase.from_uri("sqlite:///data/databases/company.db")

print(f"table names {db.get_usable_table_names()}")
print(f"Table info {db.get_table_info()}")

table names ['Employees', 'Projects']
Table info 
CREATE TABLE "Employees" (
	id INTEGER, 
	name TEXT NOT NULL, 
	role TEXT NOT NULL, 
	department TEXT NOT NULL, 
	salary REAL NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from Employees table:
id	name	role	department	salary
1	John Smith	Software Engineer	Engineering	75000.0
2	Sarah Johnson	Product Manager	Product	85000.0
3	Mike Davis	Data Scientist	Analytics	80000.0
*/


CREATE TABLE "Projects" (
	id INTEGER, 
	name TEXT NOT NULL, 
	budget REAL NOT NULL, 
	status TEXT NOT NULL, 
	lead_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(lead_id) REFERENCES "Employees" (id)
)

/*
3 rows from Projects table:
id	name	budget	status	lead_id
1	E-commerce Platform	250000.0	In Progress	1
2	Mobile App Development	180000.0	Planning	2
3	Data Analytics Dashboard	120000.0	Completed	3
*/


#Custom sql database content process


In [10]:
from typing import List
from langchain_core.documents import Document


In [14]:
def sql_to_document(db_path: str) -> List[Document]:
    db = sqlite3.connect(db_path)
    cursor = db.cursor()
    documents = []
    #step 1 : create a document for each table
    cursor.execute("SELECT name from sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    
    for table in tables:
        table_name = table[0]

        # Get table schema (column info)
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        column_names = [col[1] for col in columns]

        # Get table data
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()

        # Create table overview document
        table_content = f"Table: {table_name}\n"
        table_content += f"Columns: {', '.join(column_names)}\n"
        table_content += f"Total Records: {len(rows)}\n\n"
        
         # Add sample records (first 5 rows)
        table_content += "Sample Records:\n"
        for row in rows[:5]:
            record = dict(zip(column_names, row))
            table_content += f"{record}\n"

        # Create LangChain Document
        doc = Document(
            page_content=table_content,
            metadata={
                'source': db_path,
                'table_name': table_name,
                'num_records': len(rows),
                'data_type': 'sql_table'
            }
        )

        documents.append(doc)
        
        return documents

In [19]:
docs = sql_to_document("data/databases/company.db")
docs[0].page_content


"Table: Employees\nColumns: id, name, role, department, salary\nTotal Records: 10\n\nSample Records:\n{'id': 1, 'name': 'John Smith', 'role': 'Software Engineer', 'department': 'Engineering', 'salary': 75000.0}\n{'id': 2, 'name': 'Sarah Johnson', 'role': 'Product Manager', 'department': 'Product', 'salary': 85000.0}\n{'id': 3, 'name': 'Mike Davis', 'role': 'Data Scientist', 'department': 'Analytics', 'salary': 80000.0}\n{'id': 4, 'name': 'Emily Brown', 'role': 'UX Designer', 'department': 'Design', 'salary': 70000.0}\n{'id': 5, 'name': 'David Wilson', 'role': 'DevOps Engineer', 'department': 'Engineering', 'salary': 78000.0}\n"